In [1]:
import sys
sys.path.append("../")
import nanonis
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import xlrd
import lmfit
from lmfit import Model
import glob
import matplotlib.colors as colors
from scipy import constants as const
from matplotlib.widgets import Button, Slider
%matplotlib qt
%reload_ext autoreload
%autoreload 2
spectra=nanonis.biasSpectroscopy()
path1="//sannet01.nanogune.intranet/CIC09/STMDATA/Berlino/Pb(111)/2023-11-20 - Pb on graphene/2023-11-30 - Pb on Gr-SiC0001 - sandra2"

# Theory program

In [2]:
def bcs(delta,x,eta):
    if delta<=0.01: 
        return 1
    else:
        return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta**2-(x+eta*1j)**2)))
    
def bcs_i(delta,x,eta,n):
    if int(np.mod(np.abs(np.around(n)),2))==0:
        if delta<=0.01: 
            return 1
        else:
            return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta**2-(x+eta*1j)**2)))
    else:
        return 1

def fermi(T,x):
    if T==0.0:
        return np.heaviside(-x,1)
    else:
        return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0,Delta):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0,Delta):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1D(V,R1,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta):
    x=np.linspace(-4*np.max(V),4*np.max(V),50000)
    a,b=np.meshgrid(x,E1(C1,C2,V,n,Q0,Delta))
    t=a+b
    return np.dot( bcs_i(Delta,t,eta,n+Q0)*fermi(T,-t),fermi(T,x) )/R1

def Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta):
    x=np.linspace(-4*np.max(V),4*np.max(V),50000)
    a,b=np.meshgrid(x,E2(C1,C2,V,-n,-Q0,Delta))
    t=a+b
    return np.dot((fermi(T,-t)),bcs_i(Delta,x,eta,n+Q0)*fermi(T,x))/R2



def PND(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    mn=(Gamma1D(V,R1,C1,C2,-n,Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(-V,R2,C1,C2,n,-Q0,Delta,delta_t,delta_s,T,eta))/(Gamma1D(-V,R1,C1,C2,n+1,-Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(V,R2,C1,C2,-n-1,Q0,Delta,delta_t,delta_s,T,eta))
    en=(Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta))/(Gamma1D(V,R1,C1,C2,n+1,Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(-V,R2,C1,C2,-n-1,-Q0,Delta,delta_t,delta_s,T,eta))
    return en,mn

def check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    Vmax=V[-1]
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PND(Vmax,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if pn<0.0001 and p_n<0.0001:
            #print(len(a))
            break 
    return n[-1]

def all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    a=[]
    b=[]
    for i in range(n):
        an,bn=PND(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        a.append(an)
        b.append(bn)
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(i+1):
            temp1*=a[j]     
            temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    return Gamma1D(V,R1,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta)-Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)

def G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    return Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta)-Gamma2D(-V,R2,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)

def currentD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    n=check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta)
    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n)
    I=p0*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        I+=p_n[i-1]*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,-i)
    t=np.gradient(I)
    return t/np.sum(t)


def currentD2(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    n=check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta)
    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n)
    I=p0*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        I+=p_n[i-1]*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,-i)
    t=np.gradient(I)
    return t/np.sum(t)

## Simple understanding progam

In [22]:
def par(n,q0,Ec,delta):
    return Ec*(n-q0)**2+delta*(1-(-1)**n)/2
def find_oc(q0,Ec,delta):
    a=int(q0)
    e1=par(a-1,q0,Ec,delta)
    e2=par(a,q0,Ec,delta)
    e3=par(a+1,q0,Ec,delta)
    b=[e1,e2,e3]
    return a+b.index(min(b))-1

def oc(q0,Ec,delta,T):
    a=int(q0)
    e1=par(a-1,q0,Ec,delta)
    e2=par(a,q0,Ec,delta)
    e3=par(a+1,q0,Ec,delta)
    b=np.array([e1,e2,e3])
    Z=sum(np.exp(-b/T))
    return np.exp(-b/T)/Z,a

def exc(q0,Ec,delta,T,deltat):
    b,a=oc(q0,Ec,delta,T)
    e0=par(a-2,q0,Ec,delta)
    e1=par(a-1,q0,Ec,delta)
    e2=par(a,q0,Ec,delta)
    e3=par(a+1,q0,Ec,delta)
    e4=par(a+1,q0,Ec,delta)
    return [b[0],b[0],b[1],b[1],b[2],b[2]],[np.abs(e2-e1)+deltat,-np.abs(e1-e0)-deltat,np.abs(e3-e2)+deltat,-np.abs(e1-e2)-deltat,np.abs(e4-e3)+deltat,-np.abs(e2-e3)-deltat]

fig,ax=plt.subplots(2)

Ec=1
delta=1
q0=1.2
T=0.1
deltat=1
N=range(-5,5)
x=np.linspace(-4,4,100)
for i in N:
    ax[0].plot(x,par(i,x,Ec,delta))

n=find_oc(q0,Ec,delta)
ax[0].scatter(q0,par(n,q0,Ec,delta))
ax[0].set_ylim(0,3)
a,b=exc(q0,Ec,delta,T,deltat)
z=np.zeros(len(x))
for i in range(len(a)):
    z+=a[i]/((x-b[i])**2+0.1)

ax[1].plot(x,z)


axfreq = fig.add_axes([0.25, 0.0, 0.65, 0.03])
q0s = Slider(
    ax=axfreq,
    label='Max',
    valmin=-3,
    valmax=3,
    valinit=0,
)

axfreq = fig.add_axes([0.25, 0.03, 0.65, 0.03])
Ts = Slider(
    ax=axfreq,
    label='Max',
    valmin=0,
    valmax=2,
    valinit=0.1,
)


axamp = fig.add_axes([0.03, 0.25, 0.0225, 0.63])
Ecs = Slider(
    ax=axamp,
    label="Min",
    valmin=0,
    valmax=2,
    valinit=1,
    orientation="vertical"
)

axamp = fig.add_axes([0.01, 0.25, 0.0225, 0.63])
etas = Slider(
    ax=axamp,
    label="Min",
    valmin=0,
    valmax=1,
    valinit=0.1,
    orientation="vertical"
)

def update(val):
    ax[0].clear()
    ax[1].clear()
    for i in N:
        ax[0].plot(x,par(i,x,Ecs.val,delta))

    n=find_oc(q0s.val,Ecs.val,delta)
    ax[0].scatter(q0s.val,par(n,q0s.val,Ecs.val,delta))
    ax[0].set_ylim(0,3)
    a,b=exc(q0s.val,Ecs.val,delta,Ts.val,deltat)
    z=np.zeros(len(x))
    for i in range(len(a)):
        z+=a[i]/((x-b[i])**2+etas.val)
    ax[1].plot(x,z)
    

q0s.on_changed(update)
Ecs.on_changed(update) 
Ts.on_changed(update)  
etas.on_changed(update)  




0

# Island By Island

## 1st Island

### Data for the island

In [70]:
h=0
fig, ax = plt.subplots()
Z=[]
for i in file:
    file=glob.glob(path1+'/*SZ01231204*')
    spectra.load(i)
    x=spectra.bias
    y=spectra.conductance
    z=spectra.current
    #plt.plot(x,y/y[0]+0.2*h)
    #plt.scatter(h,z[0])
    h+=1
    Z.append(y/y[0])

val1=1
val2=0
#create the slider
ax.imshow(Z,aspect='auto',vmax=val1,vmin=val2)
fig.subplots_adjust(left=0.25, bottom=0.25)

axfreq = fig.add_axes([0.25, 0.1, 0.65, 0.03])
val1_slider = Slider(
    ax=axfreq,
    label='Max',
    valmin=0.1,
    valmax=10,
    valinit=val1,
)

# Make a vertically oriented slider to control the amplitude
axamp = fig.add_axes([0.1, 0.25, 0.0225, 0.63])
val2_slider = Slider(
    ax=axamp,
    label="Min",
    valmin=-5,
    valmax=4,
    valinit=val2,
    orientation="vertical"
)


val1_slider.on_changed(update)
val2_slider.on_changed(update)  

def update(val):
    ax.clear()
    ax.imshow(Z,aspect='auto',vmax=val1_slider.val,vmin=val2_slider.val)


In [387]:
h=0
fig, ax = plt.subplots()
Z=[]
file=glob.glob(path1+'/*SZ01231204*')
for i in file:
    spectra.load(i)
    x=spectra.bias
    y=spectra.conductance
    z=spectra.current
    y=sc.signal.savgol_filter(y/y[0],7,3)
    ax.plot(x,y+0.2*h)
    h+=1



In [205]:
h=0
#fig, ax = plt.subplots()
#fig2, ax2 = plt.subplots()
Z=[]
Jc=[]

c=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
file=glob.glob(path1+'/*IZ02y120*')
for i in [0,1,2,3,4,5,6,7,8,9]:
    spectra.load(file[i])
    x=np.array(spectra.bias)
    y=np.array(spectra.current)
    x=x*np.abs((y[0]-y[-1])/4e-8)
    z=np.array(spectra.biasVI_b)
    Z.append(spectra.z)
    #z2=spectra.biasVI_f
    n=int(len(x)/2)
    z=z[0:n]
    x=x[0:n]
    temp=-np.gradient(z)
    ax.plot(x,z,color=c[i])
    ax.plot(x,10*temp,color=c[i])
    Jc.append(x[np.where(temp==np.max(temp))[0][0]])
    print(np.abs((y[0]-y[-1])/4e-8))

R=[]
file=glob.glob(path1+'/*SZ01231204*')
for i in [0,1,2,3,4,5,6,7,8,9]:
    
    spectra.load(file[i])
    x=spectra.bias
    y=spectra.conductance
    R.append(x[0]/y[0])

plt.scatter(R,Jc)


0.9610117999999999
0.960170575
0.9546038250000001
0.9470846749999999
0.94146095
0.9350075750000001
0.9258049999999999
0.9169174250000001
0.904395675
0.899770275


In [211]:
fig, ax = plt.subplots()
Z=[]
Jc=[]

c=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
file=glob.glob(path1+'/*IZ02y120*')
Y=[]
for i in [0,1,2,3,4,5,6]:
    spectra.load(file[i])
    x=np.array(spectra.bias)
    y=np.array(spectra.current)
    x=x*np.abs((y[0]-y[-1])/4e-8)
    z=np.array(spectra.biasVI_b)
    zz=np.array(spectra.biasVI_f)
    n=int(len(x)/2)
    z1=zz[0:n-1]-zz[n-1]
    x1=x[0:n-1]
    z2=z[n:-1]-z[n]
    x2=x[n:-1]
    ax.plot(x1,(z1+np.flip(z2)),color=c[i])
    #ax.plot(z,z,color=c[i])
    


In [210]:
h=0
fig, ax = plt.subplots()
Z=[]
file=glob.glob(path1+'/*SP01231204*')
for i in [0,1,2,3,4,5,6]:
    spectra.load(file[i])
    x=spectra.bias
    z=spectra.current
    ax.plot(x,z)

## Island 2

In [367]:
file=glob.glob(path1+'*/SP2-*')
island1=file[0:400]
island2=file[402:547]
island3=file[551:-1]
iclose=[]
ifar=[]
for i in island3:
    spectra.load(i)
    a=spectra.current[0]
    if a>1e-9:
        iclose.append(i)
    else:
        ifar.append(i)
    


In [497]:
fig, ax=plt.subplots()
Z=[]
for i in range(75,252):
    spectra.load(iclose[i])
    x=np.array(spectra.bias)
    y=np.array(spectra.conductance)
    y=sc.signal.savgol_filter(y/y[0],7,3)
    Z.append(y)

val1=1
val2=0
#create the slider
ax.imshow(Z,aspect='auto',vmax=val1,vmin=val2,extent=[-5,5,0,252])
fig.subplots_adjust(left=0.25, bottom=0.25)

axfreq = fig.add_axes([0.25, 0.1, 0.65, 0.03])
val1_slider = Slider(
    ax=axfreq,
    label='Max',
    valmin=0.1,
    valmax=10,
    valinit=val1,
)

# Make a vertically oriented slider to control the amplitude
axamp = fig.add_axes([0.1, 0.25, 0.0225, 0.63])
val2_slider = Slider(
    ax=axamp,
    label="Min",
    valmin=-5,
    valmax=4,
    valinit=val2,
    orientation="vertical"
)

def update(val):
    ax.clear()
    ax.imshow(Z,aspect='auto',vmax=val1_slider.val,vmin=val2_slider.val,extent=[-5,5,0,252])

val1_slider.on_changed(update)
val2_slider.on_changed(update)  


0

In [496]:
order=[]

for i in iclose:
    spectra.load(i)
    x=np.array(spectra.bias)
    t=len(x)//2
    x=x[t-50:t+50]
    y=np.array(spectra.conductance)[t-50:t+50]
    y=sc.signal.savgol_filter(y/y[0],9,3)
    #plt.plot(x,y+h)
    a=np.where(y==max(y))
    #plt.scatter(x[a],y[a]+h)
    order.append(x[a])

order, iclose = zip(*sorted(zip(order, iclose)))
order, ifar = zip(*sorted(zip(order, ifar)))

In [498]:
fig, ax=plt.subplots()


h=0
for i in range(0,52):
    spectra.load(iclose[i])
    x=np.array(spectra.bias)[0:200]
    y=np.array(spectra.conductance)[0:200]
    y=sc.signal.savgol_filter(y/y[0],15,3)
    plt.plot(1000*x,y+0.1*h)
    a=sc.signal.find_peaks(y, height=None, threshold=None, distance=None, prominence=None, width=3, wlen=None, rel_height=0.2, plateau_size=None)[0]
    #a=np.where(y==max(y))
    #ax.scatter(1000*x[a],y[a]/y[a]+h)
    h+=1

#h=0
#for i in range(0,252):
#    spectra.load(iclose[i])
#    x=np.array(spectra.bias)[350:-1]
#    y=np.array(spectra.conductance)[350:-1]
#    y=sc.signal.savgol_filter(y/y[-1],15,3)
#    #plt.plot(1000*x,y+h)
#    a=sc.signal.find_peaks(y, height=None, threshold=None, distance=None, prominence=None, width=3, wlen=None, rel_height=0.2, plateau_size=None)[0]
#    #a=np.where(y==max(y))
#    ax.scatter(1000*x[a],y[a]/y[a]+h)
#    h+=1
#val1=3
#val2=0.1
#create the slider
#ax.imshow(Z,aspect='auto',vmax=val1,vmin=val2,extent=[-5,5,0,252])
